# Day 24: Air Duct Spelunking

## Part One
You've finally met your match; the doors that provide access to the roof are locked tight, and all of the controls and related electronics are inaccessible. You simply can't reach them.

The robot that cleans the air ducts, however, can.

It's not a very fast little robot, but you reconfigure it to be able to interface with some of the exposed wires that have been routed through the HVAC system. If you can direct it to each of those locations, you should be able to bypass the security controls.

You extract the duct layout for this area from some blueprints you acquired and create a map with the relevant locations marked (your puzzle input). `0` is your current location, from which the cleaning robot embarks; the other numbers are (in no particular order) the locations the robot needs to visit at least once each. Walls are marked as `#`, and open passages are marked as `.`. Numbers behave like open passages.

For example, suppose you have a map like the following:

```
###########
#0.1.....2#
#.#######.#
#4.......3#
###########
```

To reach all of the points of interest as quickly as possible, you would have the robot take the following path:

* `0` to `4` (2 steps)

* `4` to `1` (4 steps; it can't move diagonally)

* `1` to `2` (6 steps)

* `2` to `3` (2 steps)

Since the robot isn't very fast, you need to find it the shortest route. This path is the fewest steps (in the above example, a total of `14`) required to start at `0` and then visit every other location at least once.

Given your actual map, and starting from location `0`, what is the fewest number of steps required to visit every non-`0` number marked on the map at least once?

---

In [1]:
import numpy as np

inputs = open('Day24.in').read().split('\n')[:-1]

In [2]:
# Initialise map, M, and position of air ducts, ducts.
M = np.zeros((len(inputs), len(inputs[0]))).astype(int)
ducts = {}

# Fill in map and find duct coords
for x, row in enumerate(inputs):
    for y, val in enumerate(row):
        M[x,y] = 1 if val == '#' else 0
        if val not in ('#', '.'):
            ducts[int(val)] = (x,y)

def legal(m, c):
    """
    Takes a map, m, and a coordinate, c, and returns the legal tuples you can move to
    """
    l = set()
    x, y = c
    
    for i in range(4):
        dx, dy = int((1j**i).real), int((1j**i).imag)
        if m[x+dx, y+dy] == 0:
            l.add((x+dx, y+dy))

    return l

def a_search(Map, start, goal):
    """
    Takes map m and coordinates start, goal and returns length between coords.
    
    Input c1 as a tuple to avoid pointer issues
    """
    cost = 0
    visited = {start}
    
    def iterate(f, v, m=Map, g=goal):
        new = set()
        for c in f:
            new |= legal(m, c) - v
        
        return new
    
    front = visited.copy()
    
    while goal not in front:
        new = iterate(front, visited)
        visited |= new
        front = new
        cost += 1
    
    return cost

# Init number of ducts and dictionary for edge lengths of graph, G
n = max(ducts.keys())+1
G = {}

# Get edge lengths (distance between ducts)
for i in range(n):
    for j in range(i+1,n):
        G[(i,j)] = G[(j,i)] = a_search(M, ducts[i], ducts[j])
        
# Solve problem
def recursive_walk(G, walk=[0], next_=set(range(1,8)), cost=0):
    """
    Takes graph edges G, starts at 0 and and finds the optimal route visiting
    each node once. It is just a simple exhaustive search but it quite quick for
    8 nodes.
    """
    if next_:
        costs = [recursive_walk(G, walk + [i], next_ - {i}, cost + G[(walk[-1],i)]) for i in next_]
        return min(costs)
    else:
        return cost, walk

# Solution
c, w = recursive_walk(G)
print(f"It takes {c} steps to traverse the shortest path visiting nodes in this order:\n{' -> '.join([str(i) for i in w])}")

It takes 500 steps to traverse the shortest path visiting nodes in this order:
0 -> 3 -> 2 -> 1 -> 5 -> 6 -> 7 -> 4


---

# Part Two
Of course, if you leave the cleaning robot somewhere weird, someone is bound to notice.

What is the fewest number of steps required to start at `0`, visit every non-`0` number marked on the map at least once, and then return to `0`?

---

In [3]:
def recursive_return(G, walk=[0], next_=set(range(1,8)), cost=0):
    """
    Takes graph edges G, starts at 0 and and finds the optimal route visiting
    each node once and returning to 0. It is just a simple exhaustive search but it 
    quite quick for 8 nodes.
    """
    if next_:
        costs = [recursive_return(G, walk + [i], next_ - {i}, cost + G[tuple(sorted((walk[-1],i)))]) for i in next_]
        return min(costs)
    else:
        return cost + G[(0,walk[-1])], walk + [0]

# Solution
c, w = recursive_return(G)
print(f"It takes {c} steps to traverse the shortest loop visiting nodes in this order:\n{' -> '.join([str(i) for i in w])}")

It takes 748 steps to traverse the shortest loop visiting nodes in this order:
0 -> 1 -> 5 -> 4 -> 7 -> 6 -> 2 -> 3 -> 0


---